<a href="https://colab.research.google.com/github/RubenGres/AerialDreams/blob/main/train_controlnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---


In [ ]:
!git clone https://github.com/huggingface/diffusers/
!git clone https://github.com/RubenGres/AerialDreams

In [9]:
%cp diffusers/examples/controlnet/train_controlnet.py train_controlnet.py
%cp diffusers/examples/controlnet/requirements.txt requirements.txt

In [ ]:
%pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117
%pip install git+https://github.com/huggingface/diffusers.git transformers accelerate xformers==0.0.16 wandb
%pip install bitsandbytes
%pip install -r requirements.txt

In [ ]:
!wandb login
!huggingface-cli login

In [ ]:
from accelerate.utils import write_basic_config
write_basic_config()

In [ ]:
import os
from glob import glob

f = open("validation/prompts.txt")
prompts = f.read().split('\n')

images = glob("validation/mask/*.png")
images = [os.path.abspath(i) for i in images]

In [ ]:
SD_MODEL = "runwayml/stable-diffusion-v1-5"
MODEL_OUT = "modelout"
DATASET = "rgres/AerialDreams"

VAL_PROMPTS = '"' + '" "'.join(prompts) + '"'
VAL_IMAGES = '"' + '" "'.join(images) + '"'
VAL_STEPS = 1000

IMG_COL = "image"
CONDITIONING_IMG_COL = "segmentation"
CAPTION = "prompt"

PROJECT_NAME = "aerialdreams"

LEARNING_RATE=4e-6 # between 1e-4 and 2e-6
BATCH_SIZE=5
EPOCHS=3
CKPT_STEPS=2000

In [ ]:
!python train_controlnet.py

In [ ]:
cmd = f"""accelerate launch train_controlnet.py \
 --pretrained_model_name_or_path={SD_MODEL} \
 --output_dir={MODEL_OUT} \
 --dataset_name={DATASET} \
 --conditioning_image_column={CONDITIONING_IMG_COL} \
 --image_column={IMG_COL} \
 --caption_column={CAPTION} \
 --resolution=512 \
 --learning_rate={LEARNING_RATE} \
 --validation_image {VAL_IMAGES} \
 --validation_prompt {VAL_PROMPTS} \
 --validation_steps={VAL_STEPS} \
 --train_batch_size={BATCH_SIZE} \
 --num_train_epochs={EPOCHS} \
 --tracker_project_name={PROJECT_NAME} \
 --enable_xformers_memory_efficient_attention \
 --checkpointing_steps={CKPT_STEPS} \
 --report_to wandb \
 --push_to_hub
"""

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

text_file = open("launch_train.sh", "w")
text_file.write(cmd)
text_file.close()

!chmod +x launch_train.sh

In [ ]:
!./launch_train.sh